Anne Chel, 10727477
Tyler Cools, 11004851
Kyle Molenaar, 10779027

# Fuzzy logic model for price classification of used cars

### Hierachie
- system 1: 
    * input: number of kilometers, age of the car.
    * output: price
- system 2:
    * input: gearbox, horsepower.
    * output: price
- system 3: 
    * input: fueltype, vehicletype.
    * output: price
- system 4:
    * input: combined output of system 1, 2 and 3, brand of the car.
    * output: price

The <b>fuzzifier</b> needs:
- input variables, output variables, described above. 
- membership functions for its variables, in this project three are used: triangular fuzzy logic membership functions, trapezoidal fuzzy logic membership function and singleton membership functions. 

The <b>rules</b> include:
- an antecedent containing a selected mf per variable (e.g. if x1 is low ...)
- operators: and, or, not (only consider one operator in a rule's antecedent)
- a consequent containing a selected mf for the output variable (Mamdani) or a formula for the output variable (TSK)

The <b>inference mechanism</b> does:
- uses the firing strength of a rule to calculate the result of the implication
- (for which it uses an implication operator: min, prod)
- aggregates the results

The aggregated result is then <b>defuzzified</b> and becomes a crisp value.


In [1]:
# Membership functions 

import math
import numpy as np
from collections import defaultdict, Counter
from __future__ import division

class TriangularMF:
    """Triangular fuzzy logic membership function class."""
    def __init__(self, name, start, top, end):
        self.name = name
        self.start = start
        self.top = top
        self.end = end

    def calculate_membership(self, x):
        if x <= self.start:
            return 0
        elif x >= self.start and x <= self.top:
            return (x-self.start)/(self.top-self.start)
        elif x >= self.top and x <= self.end:
            return (self.end-x)/(self.end-self.top)
        elif x >= self.end:
            return 0
        
class TrapezoidalMF:
    """Trapezoidal fuzzy logic membership function class."""
    def __init__(self, name, start, left_top, right_top, end):
        self.name = name
        self.start = start
        self.left_top = left_top
        self.right_top = right_top
        self.end = end

    def calculate_membership(self, x):
        if x <= self.start:
            return 0
        elif x >= self.start and x <= self.left_top:
            return (x-self.start)/(self.left_top-self.start)
        elif x >= self.left_top and x <= self.right_top:
            return 1
        elif x >= self.right_top and x <= self.end:
            return (self.end-x)/(self.end-self.right_top)
        elif x >= self.end:
            return 0

class SingletonMF:
    """Singleton fuzzy locgic membership function class."""
    def __init__(self, name, value):
        self.name = name
        self.value = value
    
    def calculate_membership(self, x):
        if x == self.value:
            return 1
        else:
            return 0

In [2]:
class Variable:
    """General class for variables in an FLS."""
    def __init__(self, name, range, mfs):
        self.name = name
        self.range = range
        self.mfs = mfs

    def calculate_memberships(self, x):
        """Test function to check whether
        you put together the right mfs in your variables."""
        return {
            mf.name : mf.calculate_membership(x)
            for mf in self.mfs
        }

    def get_mf_by_name(self, name):
        for mf in self.mfs:
            if mf.name == name:
                return mf

class Input(Variable):
    """Class for input variables, inherits 
    variables and functions from superclass Variable."""
    def __init__(self, name, range, mfs):
        super().__init__(name, range, mfs)
        self.type = "input"

class Output(Variable):
    """Class for output variables, inherits 
    variables and functions from superclass Variable."""
    def __init__(self, name, range, mfs):
        super().__init__(name, range, mfs)
        self.type = "output"

In [3]:
# Input variables for used car system.

# fuelType ['diesel', 'benzin', 'lpg', 'hybrid', 'elektro']
fuelType_mf1 = SingletonMF("diesel", 1)
fuelType_mf2 = SingletonMF("benzin", 2)
fuelType_mf3 = SingletonMF("lpg", 3)
fuelType_mf4 = SingletonMF("cng", 4)
mfs_fuelType = [fuelType_mf1, fuelType_mf2, fuelType_mf3, fuelType_mf4]
fuelType = Input("fuelType", (1, 4), mfs_fuelType)

# yearOfRegistration [min = 1910,  max = 2018]
yearOfRegistration_mf1 = TrapezoidalMF("really old", 1909, 1910, 1940, 1964)
yearOfRegistration_mf2 = TriangularMF("very old", 1960, 1968, 1971)
yearOfRegistration_mf3 = TriangularMF("old", 1965, 1975, 1978)
yearOfRegistration_mf4 = TriangularMF("bit old", 1973, 1982, 1985)
yearOfRegistration_mf5 = TriangularMF("average", 1982, 1989, 1992)
yearOfRegistration_mf6 = TriangularMF("bit new", 1990, 1996, 1999)
yearOfRegistration_mf7 = TriangularMF("new", 1995, 2003, 2006)
yearOfRegistration_mf8 = TriangularMF("very new", 2004, 2010, 2013)
yearOfRegistration_mf9 = TrapezoidalMF("really new", 2010, 2017, 2018, 2019)
mfs_yearOfRegistration = [yearOfRegistration_mf1, yearOfRegistration_mf2, yearOfRegistration_mf3, 
                          yearOfRegistration_mf4, yearOfRegistration_mf5, yearOfRegistration_mf6,
                          yearOfRegistration_mf7,yearOfRegistration_mf8, yearOfRegistration_mf9 ]
yearOfRegistration = Input("yearOfRegistration", (1910, 2018), mfs_yearOfRegistration)

# kilometer [min = 5000, max = 150000]
kilometer_mf1 = TriangularMF("really low", 0, 5000, 15000)
kilometer_mf2 = TriangularMF("very low", 14000, 20000, 30000)
kilometer_mf3 = TriangularMF("low", 29000, 35000, 45000)
kilometer_mf4 = TriangularMF("bit low", 44000, 50000, 60000)
kilometer_mf5 = TriangularMF("average", 59000, 65000, 75000)
kilometer_mf6 = TriangularMF("bit high", 69000, 85000, 90000)
kilometer_mf7 = TriangularMF("high", 80000, 100000, 105000)
kilometer_mf8 = TriangularMF("very high", 100000, 130000, 120000)
kilometer_mf9 = TriangularMF("really high", 110000, 120000, 130000)
kilometer_mf10 = TriangularMF("highest", 120000, 140000, 150001)
mfs_kilometer = [kilometer_mf1, kilometer_mf2, kilometer_mf3, 
                kilometer_mf4, kilometer_mf5, kilometer_mf6,
                kilometer_mf7, kilometer_mf8, kilometer_mf9, 
                kilometer_mf10]
kilometer = Input("kilometer", (5000, 150000), mfs_kilometer)

# gearbox ['automatik', 'manuell']
gearbox_mf1 = SingletonMF("auto", 1)
gearbox_mf2 = SingletonMF("man", 2)
mfs_gearbox = [gearbox_mf1, gearbox_mf2]
gearbox = Input("gearbox", (1, 2), mfs_gearbox)

# vehicleType ['suv', 'kleinwagen', 'limousine', 'cabrio', 'bus', 'combi', 'coupe', 'andere']
vehicleType_mf4 = SingletonMF("suv", 4)
vehicleType_mf1 = SingletonMF("kleinwagen", 1)
vehicleType_mf5 = SingletonMF("limousin", 5)
vehicleType_mf3 = SingletonMF("bus", 3)
vehicleType_mf2 = SingletonMF("combi", 2)
mfs_vehicleType = [vehicleType_mf1, vehicleType_mf2, vehicleType_mf3, vehicleType_mf4, vehicleType_mf5]
vehicleType = Input("vehicleType", (1, 5), mfs_vehicleType)

# horsePower [min = 21, max = 776]
horsePower_mf1 = TrapezoidalMF("ultra low", 20, 21, 30, 50)
horsePower_mf2 = TriangularMF("low", 45, 90, 100)
horsePower_mf3 = TriangularMF("bit low", 90, 125, 150)
horsePower_mf4 = TriangularMF("medium", 140, 175, 200)
horsePower_mf5 = TriangularMF("bit high", 190, 225, 250)
horsePower_mf6 = TriangularMF("high", 240, 275, 300)
horsePower_mf7 = TriangularMF("ultra high", 290, 325, 350)
horsePower_mf8 = TrapezoidalMF("highest", 340, 500, 776, 777)
mfs_horsePower = [horsePower_mf1, horsePower_mf2, horsePower_mf3, horsePower_mf4, horsePower_mf5,
                 horsePower_mf6, horsePower_mf7, horsePower_mf8]
horsePower = Input("horsePower", (21, 776), mfs_horsePower)

# similar cars are grouped together
brand_mf1 = SingletonMF("cat1", 1)
brand_mf2 = SingletonMF("cat2", 2)
brand_mf3 = SingletonMF("cat3", 3)
brand_mf4 = SingletonMF("cat4", 4)
brand_mf5 = SingletonMF("cat5", 5)
brand_mf6 = SingletonMF("cat6", 6)
brand_mf7 = SingletonMF("cat7", 7)
brand_mf8 = SingletonMF("cat8", 8)

mfs_brand = [brand_mf1, brand_mf2, brand_mf3, brand_mf4, brand_mf5, brand_mf6, brand_mf7, brand_mf8]
brand = Input("brand", (1, 8), mfs_brand)

all_price_mf1 = SingletonMF("ultra cheap", 1)
all_price_mf2 = SingletonMF("bit cheap", 2)
all_price_mf3 = SingletonMF("cheap", 3)
all_price_mf4 = SingletonMF("medium", 4)
all_price_mf5 = SingletonMF("bit expensive", 5)
all_price_mf6 = SingletonMF("expensive", 6)
all_price_mf7 = SingletonMF("ultra expensive", 7)

mfs_all_price = [all_price_mf1, all_price_mf2, all_price_mf3, all_price_mf4, 
                 all_price_mf5, all_price_mf6, all_price_mf7]
allPrices = Input("allPrices", (1, 7), mfs_all_price)

In [4]:
# Output variable for used car system.

# price [min = 101, max = 249000]
price_mf1 = TriangularMF("ultra cheap", 499, 1500, 3000)
price_mf2 = TriangularMF("bit cheap",3000, 4000, 6000)
price_mf3 = TriangularMF("cheap", 6000, 6500, 9000)
price_mf4 = TriangularMF("medium", 9000, 9000, 12000)
price_mf5 = TriangularMF("bit expensive", 12000, 14000 , 18000)
price_mf6 = TriangularMF("expensive", 18000, 22000, 28000)
price_mf7 = TriangularMF("ultra expensive", 28000, 60000, 100000)
mfs_price = [price_mf1, price_mf2, price_mf3, price_mf4, price_mf5, price_mf6, price_mf7]
price = Output("price", (500, 10000), mfs_price)


In [5]:
# Determine firing strength of all rules
class Rule:
    """Fuzzy rule class, initialized with an antecedent (list of strings),
    operator (string) and consequent (string)."""
    def __init__(self, n, antecedent, operator, consequent):
        self.number = n
        self.antecedent = antecedent
        self.operator = operator
        self.consequent = consequent
        self.firing_strength = 0

    def calculate_firing_strength(self, datapoint, inputs):
        sets = []
        for n in range(len(inputs)):
            x = datapoint[n]
            mfName = self.antecedent[n]
            mf = inputs[n].get_mf_by_name(mfName)
            fuzzySet = mf.calculate_membership(x)
            sets.append(fuzzySet)
        if self.operator == "and":
            self.firing_strength = min(sets)
        elif self.operator == "or":
            self.firing_strength = max(sets)
        return self.firing_strength

In [6]:
# Return largest firing strengths encountered
from collections import Counter

class Rulebase:
    """The fuzzy rulebase collects all rules for the FLS, can
    calculate the firing strengths of its rules."""
    def __init__(self, rules):
        self.rules = rules

    def calculate_firing_strengths(self, datapoint, inputs):
        result = Counter()
        for i, rule in enumerate(self.rules):
            fs = rule.calculate_firing_strength(datapoint, inputs)
            consequent = rule.consequent
            if fs > result[consequent]:
                result[consequent] = fs
        return result

In [7]:
# Rules

# year/km
rules_1 = [Rule(1, ["really low", "really old"], "and", "bit cheap"), 
           Rule(2, ["really low", "very old"], "and", "bit cheap"),
           Rule(3, ["really low", "old"], "and", "bit cheap"), 
           Rule(4, ["really low", "bit old"], "and", "medium"),
           Rule(5, ["really low", "average"], "and", "medium"), 
           Rule(6, ["really low", "bit new"], "and", "bit expensive"),
           Rule(7, ["really low", "new"], "and", "bit expensive"), 
           Rule(8, ["really low", "very new"], "and", "expensive"), 
           Rule(9, ["really low", "really new"], "and", "ultra expensive"),
           Rule(10, ["very low", "really old"], "and", "cheap"), 
           Rule(11, ["very low", "very old"], "and", "bit cheap"),
           Rule(12, ["very low", "old"], "and", "bit cheap"), 
           Rule(13, ["very low", "bit old"], "and", "bit cheap"),
           Rule(14, ["very low", "average"], "and", "medium"), 
           Rule(15, ["very low", "bit new"], "and", "mdcium"),
           Rule(16, ["very low", "new"], "and", "bit expensive"), 
           Rule(17, ["very low", "very new"], "and", "bit expensive"), 
           Rule(18, ["very low", "really new"], "and", "expensive"),
           Rule(19, ["low", "really old"], "and", "cheap"), 
           Rule(20, ["low", "very old"], "and", "bit cheap"),
           Rule(21, ["low", "old"], "and", "bit cheap"), 
           Rule(22, ["low", "bit old"], "and", "bit cheap"),
           Rule(23, ["low", "average"], "and", "medium"), 
           Rule(24, ["low", "bit new"], "and", "medium"),
           Rule(25, ["low", "new"], "and", "medium"), 
           Rule(26, ["low", "very new"], "and", "bit expensive"), 
           Rule(27, ["low", "really new"], "and", "bit expensive"),
           Rule(28, ["bit low", "really old"], "and", "cheap"), 
           Rule(29, ["bit low", "very old"], "and", "cheap"),
           Rule(30, ["bit low", "old"], "and", "bit cheap"), 
           Rule(31, ["bit low", "bit old"], "and", "bit cheap"),
           Rule(32, ["bit low", "average"], "and", "medium"), 
           Rule(33, ["bit low", "bit new"], "and", "medium"),
           Rule(34, ["bit low", "new"], "and", "medium"), 
           Rule(35, ["bit low", "very new"], "and", "medium"), 
           Rule(36, ["bit low", "really new"], "and", "bit expensive"),
           Rule(37, ["average", "really old"], "and", "cheap"), 
           Rule(38, ["average", "very old"], "and", "cheap"),
           Rule(39, ["average", "old"], "and", "cheap"), 
           Rule(40, ["average", "bit old"], "and", "bit cheap"),
           Rule(41, ["average", "average"], "and", "bit cheap"), 
           Rule(42, ["average", "bit new"], "and", "medium"),
           Rule(43, ["average", "new"], "and", "medium"), 
           Rule(44, ["average", "very new"], "and", "medium"), 
           Rule(45, ["average", "really new"], "and", "medium"),
           Rule(46, ["bit high", "really old"], "and", "ultra cheap"), 
           Rule(47, ["bit high", "very old"], "and", "cheap"),
           Rule(48, ["bit high", "old"], "and", "cheap"), 
           Rule(49, ["bit high", "bit old"], "and", "cheap"),
           Rule(50, ["bit high", "average"], "and", "cheap"), 
           Rule(51, ["bit high", "bit new"], "and", "bit cheap"),
           Rule(52, ["bit high", "new"], "and", "bit cheap"), 
           Rule(53, ["bit high", "very new"], "and", "bit cheap"), 
           Rule(54, ["bit high", "really new"], "and", "medium"),
           Rule(55, ["high", "really old"], "and", "ultra cheap"), 
           Rule(56, ["high", "very old"], "and", "ultra cheap"),
           Rule(57, ["high", "old"], "and", "ultra cheap"), 
           Rule(58, ["high", "bit old"], "and", "cheap"),
           Rule(59, ["high", "average"], "and", "cheap"), 
           Rule(60, ["high", "bit new"], "and", "cheap"),
           Rule(61, ["high", "new"], "and", "bit cheap"), 
           Rule(62, ["high", "very new"], "and", "bit cheap"), 
           Rule(63, ["high", "really new"], "and", "bit cheap"),        
           Rule(64, ["very high", "really old"], "and", "ultra cheap"), 
           Rule(65, ["very high", "very old"], "and", "ultra cheap"),
           Rule(66, ["very high", "old"], "and", "ultra cheap"), 
           Rule(67, ["very high", "bit old"], "and", "ultra cheap"),
           Rule(68, ["very high", "average"], "and", "cheap"), 
           Rule(69, ["very high", "bit new"], "and", "cheap"),
           Rule(70, ["very high", "new"], "and", "cheap"), 
           Rule(71, ["very high", "very new"], "and", "bit cheap"), 
           Rule(72, ["very high", "really new"], "and", "bit cheap"),
           Rule(73, ["really high", "really old"], "and", "ultra cheap"), 
           Rule(74, ["really high", "very old"], "and", "ultra cheap"),
           Rule(75, ["really high", "old"], "and", "ultra cheap"), 
           Rule(76, ["really high", "bit old"], "and", "ultra cheap"),
           Rule(77, ["really high", "average"], "and", "ultra cheap"), 
           Rule(78, ["really high", "bit new"], "and", "cheap"),
           Rule(79, ["really high", "new"], "and", "cheap"), 
           Rule(80, ["really high", "very new"], "and", "cheap"), 
           Rule(81, ["really high", "really new"], "and", "bit cheap"), 
           Rule(82, ["highest", "really old"], "and", "ultra cheap"), 
           Rule(83, ["highest", "very old"], "and", "ultra cheap"),
           Rule(84, ["highest", "old"], "and", "ultra cheap"), 
           Rule(85, ["highest", "bit old"], "and", "ultra cheap"),
           Rule(86, ["highest", "average"], "and", "ultra cheap"), 
           Rule(87, ["highest", "bit new"], "and", "ultra cheap"),
           Rule(88, ["highest", "new"], "and", "cheap"), 
           Rule(89, ["highest", "very new"], "and", "cheap"), 
           Rule(90, ["highest", "really new"], "and", "cheap")  
          ]

# gearbox/horsepower
rules_2 = [Rule(1, ["auto", "ultra low"], "and", "ultra cheap"), 
           Rule(2, ["auto", "low"], "and", "cheap"),
           Rule(3, ["auto", "bit low"], "and", "medium"), 
           Rule(4, ["auto", "medium"], "and", "medium"),
           Rule(5, ["auto", "bit high"], "and", "bit expensive"), 
           Rule(6, ["auto", "high"], "and", "expensive"),
           Rule(7, ["auto", "ultra high"], "and", "expensive"), 
           Rule(8, ["auto", "highest"], "and", "ultra expensive"), 
           
           Rule(9, ["man", "ultra low"], "and", "ultra cheap"), 
           Rule(10, ["man", "low"], "and", "ultra cheap"),
           Rule(11, ["man", "bit low"], "and", "bit cheap"), 
           Rule(12, ["man", "medium"], "and", "medium"),
           Rule(13, ["man", "bit high"], "and", "medium"), 
           Rule(14, ["man", "high"], "and", "bit expensive"),
           Rule(15, ["man", "ultra high"], "and", "bit expensive"), 
           Rule(16, ["man", "highest"], "and", "expensive")
          ]

# fuel/vehicle-type
rules_3 = [ Rule(1, ["diesel", "kleinwagen"], "and", "bit cheap"), 
            Rule(2, ["diesel", "combi"], "and", "medium"),
            Rule(3, ["diesel", "bus"], "and", "medium"), 
            Rule(4, ["diesel", "suv"], "and", "bit expensive"),
            Rule(5, ["diesel", "limousin"], "and", "bit expensive"), 
            Rule(6, ["lpg", "kleinwagen"], "and", "ultra cheap"),
            Rule(7, ["lpg", "combi"], "and", "cheap"), 
            Rule(8, ["lpg", "bus"], "and", "cheap"), 
            Rule(9, ["lpg", "suv"], "and", "cheap"),
            Rule(10, ["lpg", "limousin"], "and", "cheap"), 
            Rule(11, ["cng", "kleinwagen"], "and", "ultra cheap"), 
            Rule(12, ["cng", "combi"], "and", "ultra cheap"),
            Rule(13, ["cng", "bus"], "and", "ultra cheap"), 
            Rule(14, ["cng", "suv"], "and", "cheap"),
            Rule(15, ["cng", "limousin"], "and", "cheap"), 
            Rule(16, ["benzin", "kleinwagen"], "and", "medium"),
            Rule(17, ["benzin", "combi"], "and", "bit expensive"), 
            Rule(18, ["benzin", "bus"], "and", "bit expensive"), 
            Rule(19, ["benzin", "suv"], "and", "expensive"),
            Rule(20, ["benzin", "limousin"], "and", "expensive")   
          ]

# price1/price2/price3-categorie
# price1/price2/price3-categorie
rules_4 = [Rule(1, ["ultra cheap", "ultra cheap", "ultra cheap"], "and", "ultra cheap"),
           Rule(2, ["ultra cheap", "cheap", "ultra cheap"], "and", "ultra cheap"),
           Rule(3, ["ultra cheap", "bit cheap", "ultra cheap"], "and", "cheap"), 
           Rule(4, ["ultra cheap", "medium", "ultra cheap"], "and", "cheap"),
           Rule(5, ["ultra cheap", "bit expensive", "ultra cheap"], "and", "cheap"),
           Rule(6, ["ultra cheap", "expensive", "ultra cheap"], "and", "bit cheap"),
           Rule(7, ["ultra cheap", "ultra expensive", "ultra cheap"], "and", "bit cheap"),
           Rule(8, ["ultra cheap", "ultra cheap", "cheap"], "and", "ultra cheap"),
           Rule(9, ["ultra cheap", "cheap", "cheap"], "and", "cheap"),
           Rule(10, ["ultra cheap", "bit cheap", "cheap"], "and", "cheap"), 
           Rule(11, ["ultra cheap", "medium", "cheap"], "and", "cheap"),
           Rule(12, ["ultra cheap", "bit expensive", "cheap"], "and", "bit cheap"),
           Rule(13, ["ultra cheap", "expensive", "cheap"], "and", "bit cheap"),
           Rule(14, ["ultra cheap", "ultra expensive", "cheap"], "and", "medium"),
           Rule(15, ["ultra cheap", "ultra cheap", "bit cheap"], "and", "ultra cheap"),
           Rule(16, ["ultra cheap", "cheap", "bit cheap"], "and", "cheap"),
           Rule(17, ["ultra cheap", "bit cheap", "bit cheap"], "and", "cheap"), 
           Rule(18, ["ultra cheap", "medium", "bit cheap"], "and", "bit cheap"),
           Rule(19, ["ultra cheap", "bit expensive", "bit cheap"], "and", "bit cheap"),
           Rule(20, ["ultra cheap", "expensive", "bit cheap"], "and", "medium"),
           Rule(21, ["ultra cheap", "ultra expensive", "bit cheap"], "and", "medium"),
           Rule(22, ["ultra cheap", "ultra cheap", "medium"], "and", "cheap"),
           Rule(23, ["ultra cheap", "cheap", "medium"], "and", "cheap"),
           Rule(24, ["ultra cheap", "bit cheap", "medium"], "and", "bit cheap"), 
           Rule(25, ["ultra cheap", "medium", "medium"], "and", "bit cheap"),
           Rule(26, ["ultra cheap", "bit expensive", "medium"], "and", "medium"),
           Rule(27, ["ultra cheap", "expensive", "medium"], "and", "medium"),
           Rule(28, ["ultra cheap", "ultra expensive", "medium"], "and", "medium"),
           Rule(29, ["ultra cheap", "ultra cheap", "bit expensive"], "and", "cheap"),
           Rule(30, ["ultra cheap", "cheap", "bit expensive"], "and", "bit cheap"),
           Rule(31, ["ultra cheap", "bit cheap", "bit expensive"], "and", "bit cheap"), 
           Rule(32, ["ultra cheap", "medium", "bit expensive"], "and", "medium"),
           Rule(33, ["ultra cheap", "bit expensive", "bit expensive"], "and", "medium"),
           Rule(34, ["ultra cheap", "expensive", "bit expensive"], "and", "bit expensive"),
           Rule(35, ["ultra cheap", "ultra expensive", "bit expensive"], "and", "bit expensive"),
           Rule(36, ["ultra cheap", "ultra cheap", "expensive"], "and", "bit cheap"),
           Rule(37, ["ultra cheap", "cheap", "expensive"], "and", "bit cheap"),
           Rule(38, ["ultra cheap", "bit cheap", "expensive"], "and", "medium"), 
           Rule(39, ["ultra cheap", "medium", "expensive"], "and", "medium"),
           Rule(40, ["ultra cheap", "bit expensive", "expensive"], "and", "medium"),
           Rule(41, ["ultra cheap", "expensive", "expensive"], "and", "bit expensive"),
           Rule(42, ["ultra cheap", "ultra expensive", "expensive"], "and", "bit expensive"),
           Rule(43, ["ultra cheap", "ultra cheap", "ultra expensive"], "and", "bit cheap"),
           Rule(44, ["ultra cheap", "cheap", "ultra expensive"], "and", "medium"),
           Rule(45, ["ultra cheap", "bit cheap", "ultra expensive"], "and", "medium"), 
           Rule(46, ["ultra cheap", "medium", "ultra expensive"], "and", "medium"),
           Rule(47, ["ultra cheap", "bit expensive", "ultra expensive"], "and", "bit expensive"),
           Rule(48, ["ultra cheap", "expensive", "ultra expensive"], "and", "bit expensive"),
           Rule(49, ["ultra cheap", "ultra expensive", "ultra expensive"], "and", "expensive"),
           Rule(50, ["cheap", "ultra cheap", "ultra cheap"], "and", "ultra cheap"),
           Rule(51, ["cheap", "cheap", "ultra cheap"], "and", "cheap"),
           Rule(52, ["cheap", "bit cheap", "ultra cheap"], "and", "cheap"), 
           Rule(53, ["cheap", "medium", "ultra cheap"], "and", "cheap"),
           Rule(54, ["cheap", "bit expensive", "ultra cheap"], "and", "bit cheap"),
           Rule(55, ["cheap", "expensive", "ultra cheap"], "and", "cheap"),
           Rule(56, ["cheap", "ultra expensive", "ultra cheap"], "and", "medium cheap"),
           Rule(57, ["cheap", "ultra cheap", "cheap"], "and", "cheap"),
           Rule(58, ["cheap", "cheap", "cheap"], "and", "cheap"),
           Rule(59, ["cheap", "bit cheap", "cheap"], "and", "cheap"), 
           Rule(60, ["cheap", "medium", "cheap"], "and", "bit cheap"),
           Rule(61, ["cheap", "bit expensive", "cheap"], "and", "bit cheap"),
           Rule(62, ["cheap", "expensive", "cheap"], "and", "medium"),
           Rule(63, ["cheap", "ultra expensive", "cheap"], "and", "medium"),
           
           Rule(64, ["cheap", "ultra cheap", "bit cheap"], "and", "cheap"),
           Rule(65, ["cheap", "cheap", "bit cheap"], "and", "cheap"),
           Rule(66, ["cheap", "bit cheap", "bit cheap"], "and", "bit cheap"), 
           Rule(67, ["cheap", "medium", "bit cheap"], "and", "bit cheap"),
           Rule(68, ["cheap", "bit expensive", "bit cheap"], "and", "medium"),
           Rule(69, ["cheap", "expensive", "bit cheap"], "and", "medium"),
           Rule(70, ["cheap", "ultra expensive", "bit cheap"], "and", "medium"),
           
           Rule(71, ["cheap", "ultra cheap", "medium"], "and", "cheap"),
           Rule(72, ["cheap", "cheap", "medium"], "and", "bit cheap"),
           Rule(73, ["cheap", "bit cheap", "medium"], "and", "bit cheap"), 
           Rule(74, ["cheap", "medium", "medium"], "and", "medium"),
           Rule(75, ["cheap", "bit expensive", "medium"], "and", "medium"),
           Rule(76, ["cheap", "expensive", "medium"], "and", "medium"),
           Rule(77, ["cheap", "ultra expensive", "medium"], "and", "bit expensive"),
           
           Rule(78, ["cheap", "ultra cheap", "bit expensive"], "and", "cheap"),
           Rule(79, ["cheap", "cheap", "bit expensive"], "and", "bit cheap"),
           Rule(80, ["cheap", "bit cheap", "bit expensive"], "and", "bit cheap"), 
           Rule(81, ["cheap", "medium", "bit expensive"], "and", "medium"),
           Rule(82, ["cheap", "bit expensive", "bit expensive"], "and", "medium"),
           Rule(83, ["cheap", "expensive", "bit expensive"], "and", "bit expensive"),
           Rule(84, ["cheap", "ultra expensive", "bit expensive"], "and", "bit expensive"),
           
           Rule(85, ["cheap", "ultra cheap", "expensive"], "and", "bit cheap"),
           Rule(86, ["cheap", "cheap", "expensive"], "and", "medium"),
           Rule(87, ["cheap", "bit cheap", "expensive"], "and", "medium"), 
           Rule(88, ["cheap", "medium", "expensive"], "and", "bit expensive"),
           Rule(89, ["cheap", "bit expensive", "expensive"], "and", "bit expensive"),
           Rule(90, ["cheap", "expensive", "expensive"], "and", "bit expensive"),
           Rule(91, ["cheap", "ultra expensive", "expensive"], "and", "expensive"),
           
           Rule(92, ["cheap", "ultra cheap", "ultra expensive"], "and", "medium"),
           Rule(93, ["cheap", "cheap", "ultra expensive"], "and", "medium"),
           Rule(94, ["cheap", "bit cheap", "ultra expensive"], "and", "medium"), 
           Rule(95, ["cheap", "medium", "ultra expensive"], "and", "bit expensive"),
           Rule(96, ["cheap", "bit expensive", "ultra expensive"], "and", "bit expensive"),
           Rule(97, ["cheap", "expensive", "ultra expensive"], "and", "expensive"),
           Rule(98, ["cheap", "ultra expensive", "ultra expensive"], "and", "expensive"),
           
           Rule(99, ["bit cheap", "ultra cheap", "ultra cheap"], "and", "ultra cheap"),
           Rule(100, ["bit cheap", "cheap", "ultra cheap"], "and", "cheap"),
           Rule(101, ["bit cheap", "bit cheap", "ultra cheap"], "and", "cheap"), 
           Rule(102, ["bit cheap", "medium", "ultra cheap"], "and", "bit cheap"),
           Rule(103, ["bit cheap", "bit expensive", "ultra cheap"], "and", "bit cheap"),
           Rule(104, ["bit cheap", "expensive", "ultra cheap"], "and", "bit cheap"),
           Rule(105, ["bit cheap", "ultra expensive", "ultra cheap"], "and", "medium"),
           
           Rule(106, ["bit cheap", "ultra cheap", "cheap"], "and", "cheap"),
           Rule(107, ["bit cheap", "cheap", "cheap"], "and", "cheap"),
           Rule(108, ["bit cheap", "bit cheap", "cheap"], "and", "bit cheap"), 
           Rule(109, ["bit cheap", "medium", "cheap"], "and", "bit cheap"),
           Rule(110, ["bit cheap", "bit expensive", "cheap"], "and", "bit cheap"),
           Rule(111, ["bit cheap", "expensive", "cheap"], "and", "medium"),
           Rule(112, ["bit cheap", "ultra expensive", "cheap"], "and", "medium"),
           
           Rule(113, ["bit cheap", "ultra cheap", "bit cheap"], "and", "cheap"),
           Rule(114, ["bit cheap", "cheap", "bit cheap"], "and", "bit cheap"),
           Rule(115, ["bit cheap", "bit cheap", "bit cheap"], "and", "bit cheap"),            
           Rule(116, ["bit cheap", "medium", "bit cheap"], "and", "bit cheap"),
           Rule(117, ["bit cheap", "bit expensive", "bit cheap"], "and", "medium"),
           Rule(118, ["bit cheap", "expensive", "bit cheap"], "and", "medium"),
           Rule(119, ["bit cheap", "ultra expensive", "bit cheap"], "and", "bit expensive"),
           
           Rule(120, ["bit cheap", "ultra cheap", "medium"], "and", "bit cheap"),
           Rule(121, ["bit cheap", "cheap", "medium"], "and", "bit cheap"),
           Rule(122, ["bit cheap", "bit cheap", "medium"], "and", "bit cheap"), 
           Rule(123, ["bit cheap", "medium", "medium"], "and", "medium"),
           Rule(124, ["bit cheap", "bit expensive", "medium"], "and", "medium"),
           Rule(125, ["bit cheap", "expensive", "medium"], "and", "bit expensive"),
           Rule(126, ["bit cheap", "ultra expensive", "medium"], "and", "bit expensive"),
           
           Rule(127, ["bit cheap", "ultra cheap", "bit expensive"], "and", "bit cheap"),
           Rule(128, ["bit cheap", "cheap", "bit expensive"], "and", "bit cheap"),
           Rule(129, ["bit cheap", "bit cheap", "bit expensive"], "and", "medium"), 
           Rule(130, ["bit cheap", "medium", "bit expensive"], "and", "medium"),
           Rule(131, ["bit cheap", "bit expensive", "bit expensive"], "and", "medium"),
           Rule(132, ["bit cheap", "expensive", "bit expensive"], "and", "bit expensive"),
           Rule(133, ["bit cheap", "ultra expensive", "bit expensive"], "and", "bit expensive"),
           
           Rule(134, ["bit cheap", "ultra cheap", "expensive"], "and", "bit cheap"),
           Rule(135, ["bit cheap", "cheap", "expensive"], "and", "medium"),
           Rule(136, ["bit cheap", "bit cheap", "expensive"], "and", "medium"), 
           Rule(137, ["bit cheap", "medium", "expensive"], "and", "medium"),
           Rule(138, ["bit cheap", "bit expensive", "expensive"], "and", "bit expensive"),
           Rule(139, ["bit cheap", "expensive", "expensive"], "and", "bit expensive"),
           Rule(140, ["bit cheap", "ultra expensive", "expensive"], "and", "expensive"),
           
           Rule(141, ["bit cheap", "ultra cheap", "ultra expensive"], "and", "medium"),
           Rule(142, ["bit cheap", "cheap", "ultra expensive"], "and", "medium"),
           Rule(143, ["bit cheap", "bit cheap", "ultra expensive"], "and", "medium"), 
           Rule(144, ["bit cheap", "medium", "ultra expensive"], "and", "bit expensive"),
           Rule(145, ["bit cheap", "bit expensive", "ultra expensive"], "and", "bit expensive"),
           Rule(146, ["bit cheap", "expensive", "ultra expensive"], "and", "expensive"),
           Rule(147, ["bit cheap", "ultra expensive", "ultra expensive"], "and", "expensive"),
           
           Rule(148, ["medium", "ultra cheap", "ultra cheap"], "and", "cheap"),
           Rule(149, ["medium", "cheap", "ultra cheap"], "and", "cheap"),
           Rule(150, ["medium", "bit cheap", "ultra cheap"], "and", "bit cheap"), 
           Rule(151, ["medium", "medium", "ultra cheap"], "and", "bit cheap"),
           Rule(152, ["medium", "bit expensive", "ultra cheap"], "and", "bit cheap"),
           Rule(153, ["medium", "expensive", "ultra cheap"], "and", "medium"),
           Rule(154, ["medium", "ultra expensive", "ultra cheap"], "and", "medium"),
           
           Rule(155, ["medium", "ultra cheap", "cheap"], "and", "cheap"),
           Rule(156, ["medium", "cheap", "cheap"], "and", "bit cheap"),
           Rule(157, ["medium", "bit cheap", "cheap"], "and", "bit cheap"), 
           Rule(158, ["medium", "medium", "cheap"], "and", "medium"),
           Rule(159, ["medium", "bit expensive", "cheap"], "and", "medium"),
           Rule(160, ["medium", "expensive", "cheap"], "and", "medium"),
           Rule(161, ["medium", "ultra expensive", "cheap"], "and", "bit expensive"),
           
           Rule(162, ["medium", "ultra cheap", "bit cheap"], "and", "bit cheap"),
           Rule(163, ["medium", "cheap", "bit cheap"], "and", "bit cheap"),
           Rule(164, ["medium", "bit cheap", "bit cheap"], "and", "bit cheap"), 
           Rule(165, ["medium", "medium", "bit cheap"], "and", "medium"),
           Rule(166, ["medium", "bit expensive", "bit cheap"], "and", "medium"),
           Rule(167, ["medium", "expensive", "bit cheap"], "and", "bit expensive"),
           Rule(168, ["medium", "ultra expensive", "bit cheap"], "and", "bit expensive"),
           
           Rule(169, ["medium", "ultra cheap", "medium"], "and", "bit cheap"),
           Rule(170, ["medium", "cheap", "medium"], "and", "bit cheap"),
           Rule(171, ["medium", "bit cheap", "medium"], "and", "medium"), 
           Rule(172, ["medium", "medium", "medium"], "and", "medium"),
           Rule(173, ["medium", "bit expensive", "medium"], "and", "medium"),
           Rule(174, ["medium", "expensive", "medium"], "and", "bit expensive"),
           Rule(175, ["medium", "ultra expensive", "medium"], "and", "bit expensive"),
           
           Rule(176, ["medium", "ultra cheap", "bit expensive"], "and", "bit cheap"),
           Rule(177, ["medium", "cheap", "bit expensive"], "and", "medium"),
           Rule(178, ["medium", "bit cheap", "bit expensive"], "and", "medium"), 
           Rule(179, ["medium", "medium", "bit expensive"], "and", "medium"),
           Rule(180, ["medium", "bit expensive", "bit expensive"], "and", "bit expensive"),
           Rule(181, ["medium", "expensive", "bit expensive"], "and", "bit expensive"),
           Rule(182, ["medium", "ultra expensive", "bit expensive"], "and", "bit expensive"),
           
           Rule(183, ["medium", "ultra cheap", "expensive"], "and", "medium"),
           Rule(184, ["medium", "cheap", "expensive"], "and", "medium"),
           Rule(185, ["medium", "bit cheap", "expensive"], "and", "medium"), 
           Rule(186, ["medium", "medium", "expensive"], "and", "bit expensive"),
           Rule(187, ["medium", "bit expensive", "expensive"], "and", "bit expensive"),
           Rule(188, ["medium", "expensive", "expensive"], "and", "bit expensive"),
           Rule(189, ["medium", "ultra expensive", "expensive"], "and", "bit expensive"),
           Rule(190, ["medium", "ultra cheap", "ultra expensive"], "and", "expensive"),
           
           Rule(344, ["medium", "ultra cheap", "ultra expensive"], "and", "medium"),
           Rule(191, ["medium", "cheap", "ultra expensive"], "and", "medium"),
           Rule(192, ["medium", "bit cheap", "ultra expensive"], "and", "bit expensive"), 
           Rule(193, ["medium", "medium", "ultra expensive"], "and", "bit expensive"),
           Rule(194, ["medium", "bit expensive", "ultra expensive"], "and", "bit expensive"),
           Rule(195, ["medium", "expensive", "ultra expensive"], "and", "expensive"),
           Rule(196, ["medium", "ultra expensive", "ultra expensive"], "and", "expensive"),
           
           Rule(197, ["bit expensive", "ultra cheap", "ultra cheap"], "and", "medium"),
           Rule(198, ["bit expensive", "cheap", "ultra cheap"], "and", "medium"),
           Rule(199, ["bit expensive", "bit cheap", "ultra cheap"], "and", "bit expensive"), 
           Rule(200, ["bit expensive", "medium", "ultra cheap"], "and", "bit expensive"),
           Rule(201, ["bit expensive", "bit expensive", "ultra cheap"], "and", "bit expensive"),
           Rule(202, ["bit expensive", "expensive", "ultra cheap"], "and", "expensive"),
           Rule(203, ["bit expensive", "ultra expensive", "ultra cheap"], "and", "expensive"),
           
           Rule(204, ["bit expensive", "ultra cheap", "cheap"], "and", "medium"),
           Rule(205, ["bit expensive", "cheap", "cheap"], "and", "medium"),
           Rule(206, ["bit expensive", "bit cheap", "cheap"], "and", "bit expensive"), 
           Rule(207, ["bit expensive", "medium", "cheap"], "and", "bit expensive"),
           Rule(208, ["bit expensive", "bit expensive", "cheap"], "and", "bit expensive"),
           Rule(209, ["bit expensive", "expensive", "cheap"], "and", "bit expensive"),
           Rule(210, ["bit expensive", "ultra expensive", "cheap"], "and", "expensive"),
           
           Rule(211, ["bit expensive", "ultra cheap", "bit cheap"], "and", "medium"),
           Rule(212, ["bit expensive", "cheap", "bit cheap"], "and", "medium"),
           Rule(213, ["bit expensive", "bit cheap", "bit cheap"], "and", "medium"), 
           Rule(214, ["bit expensive", "medium", "bit cheap"], "and", "medium"),
           Rule(215, ["bit expensive", "bit expensive", "bit cheap"], "and", "medium"),
           Rule(216, ["bit expensive", "expensive", "bit cheap"], "and", "medium"),
           Rule(217, ["bit expensive", "ultra expensive", "bit cheap"], "and", "bit expensive"),
           
           Rule(218, ["bit expensive", "ultra cheap", "medium"], "and", "medium"),
           Rule(219, ["bit expensive", "cheap", "medium"], "and", "medium"),
           Rule(220, ["bit expensive", "bit cheap", "medium"], "and", "medium"), 
           Rule(221, ["bit expensive", "medium", "medium"], "and", "medium"),
           Rule(222, ["bit expensive", "bit expensive", "medium"], "and", "bit expensive"),
           Rule(223, ["bit expensive", "expensive", "medium"], "and", "bit expensive"),
           Rule(224, ["bit expensive", "ultra expensive", "medium"], "and", "bit expensive"),
           
           Rule(225, ["bit expensive", "ultra cheap", "bit expensive"], "and", "medium"),
           Rule(226, ["bit expensive", "cheap", "bit expensive"], "and", "medium"),
           Rule(227, ["bit expensive", "bit cheap", "bit expensive"], "and", "medium"), 
           Rule(228, ["bit expensive", "medium", "bit expensive"], "and", "bit expensive"),
           Rule(229, ["bit expensive", "bit expensive", "bit expensive"], "and", "bit expensive"),
           Rule(230, ["bit expensive", "expensive", "bit expensive"], "and", "bit expensive"),
           Rule(231, ["bit expensive", "ultra expensive", "bit expensive"], "and", "expensive"),
           
           Rule(232, ["bit expensive", "ultra cheap", "expensive"], "and", "medium"),
           Rule(233, ["bit expensive", "cheap", "expensive"], "and", "medium"),
           Rule(234, ["bit expensive", "bit cheap", "expensive"], "and", "medium"), 
           Rule(235, ["bit expensive", "medium", "expensive"], "and", "bit expensive"),
           Rule(236, ["bit expensive", "bit expensive", "expensive"], "and", "bit expensive"),
           Rule(237, ["bit expensive", "expensive", "expensive"], "and", "expensive"),
           Rule(238, ["bit expensive", "ultra expensive", "expensive"], "and", "expensive"),
           
           Rule(239, ["bit expensive", "ultra cheap", "ultra expensive"], "and", "medium"),
           Rule(240, ["bit expensive", "cheap", "ultra expensive"], "and", "medium"),
           Rule(241, ["bit expensive", "bit cheap", "ultra expensive"], "and", "bit expensive"), 
           Rule(242, ["bit expensive", "medium", "ultra expensive"], "and", "bit expensive"),
           Rule(243, ["bit expensive", "bit expensive", "ultra expensive"], "and", "bit expensive"),
           Rule(244, ["bit expensive", "expensive", "ultra expensive"], "and", "expensive"),
           Rule(245, ["bit expensive", "ultra expensive", "ultra expensive"], "and", "expensive"),
           
           Rule(246, ["expensive", "ultra cheap", "ultra cheap"], "and", "cheap"),
           Rule(247, ["expensive", "cheap", "ultra cheap"], "and", "bit cheap"),
           Rule(248, ["expensive", "bit cheap", "ultra cheap"], "and", "bit cheap"), 
           Rule(249, ["expensive", "medium", "ultra cheap"], "and", "medium"),
           Rule(250, ["expensive", "bit expensive", "ultra cheap"], "and", "medium"),
           Rule(251, ["expensive", "expensive", "ultra cheap"], "and", "medium"),
           Rule(252, ["expensive", "ultra expensive", "ultra cheap"], "and", "bit expensive"),
           
           Rule(253, ["expensive", "ultra cheap", "cheap"], "and", "bit cheap"),
           Rule(254, ["expensive", "cheap", "cheap"], "and", "bit cheap"),
           Rule(255, ["expensive", "bit cheap", "cheap"], "and", "medium"), 
           Rule(256, ["expensive", "medium", "cheap"], "and", "medium"),
           Rule(257, ["expensive", "bit expensive", "cheap"], "and", "medium"),
           Rule(258, ["expensive", "expensive", "cheap"], "and", "bit expensive"),
           Rule(259, ["expensive", "ultra expensive", "cheap"], "and", "bit expensive"),
           
           Rule(260, ["expensive", "ultra cheap", "bit cheap"], "and", "bit cheap"),
           Rule(261, ["expensive", "cheap", "bit cheap"], "and", "medium"),
           Rule(262, ["expensive", "bit cheap", "bit cheap"], "and", "medium"), 
           Rule(263, ["expensive", "medium", "bit cheap"], "and", "medium"),
           Rule(264, ["expensive", "bit expensive", "bit cheap"], "and", "bit expensive"),
           Rule(265, ["expensive", "expensive", "bit cheap"], "and", "bit expensive"),
           Rule(266, ["expensive", "ultra expensive", "bit cheap"], "and", "bit expensive"),
           
           Rule(267, ["expensive", "ultra cheap", "medium"], "and", "medium"),
           Rule(268, ["expensive", "cheap", "medium"], "and", "medium"),
           Rule(269, ["expensive", "bit cheap", "medium"], "and", "medium"), 
           Rule(270, ["expensive", "medium", "medium"], "and", "bit expensive"),
           Rule(271, ["expensive", "bit expensive", "medium"], "and", "bit expensive"),
           Rule(272, ["expensive", "expensive", "medium"], "and", "bit expensive"),
           Rule(273, ["expensive", "ultra expensive", "medium"], "and", "bit expensive"),
           Rule(274, ["expensive", "ultra cheap", "bit expensive"], "and", "expensive"),
           
           Rule(275, ["expensive", "cheap", "bit expensive"], "and", "medium"),
           Rule(276, ["expensive", "bit cheap", "bit expensive"], "and", "medium"), 
           Rule(277, ["expensive", "medium", "bit expensive"], "and", "medium"),
           Rule(278, ["expensive", "bit expensive", "bit expensive"], "and", "bit expensive"),
           Rule(279, ["expensive", "expensive", "bit expensive"], "and", "bit expensive"),
           Rule(280, ["expensive", "ultra expensive", "bit expensive"], "and", "expensive"),
           Rule(281, ["expensive", "ultra cheap", "expensive"], "and", "expensive"),
           
           Rule(282, ["expensive", "cheap", "expensive"], "and", "medium"),
           Rule(283, ["expensive", "bit cheap", "expensive"], "and", "medium"), 
           Rule(284, ["expensive", "medium", "expensive"], "and", "bit expensive"),
           Rule(285, ["expensive", "bit expensive", "expensive"], "and", "bit expensive"),
           Rule(286, ["expensive", "expensive", "expensive"], "and", "expensive"),
           Rule(287, ["expensive", "ultra expensive", "expensive"], "and", "expensive"),
           Rule(288, ["expensive", "ultra cheap", "ultra expensive"], "and", "expensive"),
           
           Rule(289, ["expensive", "cheap", "ultra expensive"], "and", "medium"),
           Rule(290, ["expensive", "bit cheap", "ultra expensive"], "and", "bit expensive"), 
           Rule(291, ["expensive", "medium", "ultra expensive"], "and", "bit expensive"),
           Rule(292, ["expensive", "bit expensive", "ultra expensive"], "and", "expensive"),
           Rule(293, ["expensive", "expensive", "ultra expensive"], "and", "expensive"),
           Rule(294, ["expensive", "ultra expensive", "ultra expensive"], "and", "ultra expensive"),
           
           Rule(295, ["ultra expensive", "ultra cheap", "ultra cheap"], "and", "cheap"),
           Rule(296, ["ultra expensive", "cheap", "ultra cheap"], "and", "bit cheap"),
           Rule(297, ["ultra expensive", "bit cheap", "ultra cheap"], "and", "bit cheap"), 
           Rule(298, ["ultra expensive", "medium", "ultra cheap"], "and", "bit cheap"),
           Rule(299, ["ultra expensive", "bit expensive", "ultra cheap"], "and", "medium"),
           Rule(300, ["ultra expensive", "expensive", "ultra cheap"], "and", "medium"),
           Rule(301, ["ultra expensive", "ultra expensive", "ultra cheap"], "and", "bit expensive"),
           
           Rule(302, ["ultra expensive", "ultra cheap", "cheap"], "and", "bit cheap"),
           Rule(303, ["ultra expensive", "cheap", "cheap"], "and", "bit cheap"),
           Rule(304, ["ultra expensive", "bit cheap", "cheap"], "and", "bit cheap"), 
           Rule(305, ["ultra expensive", "medium", "cheap"], "and", "medium"),
           Rule(306, ["ultra expensive", "bit expensive", "cheap"], "and", "medium"),
           Rule(307, ["ultra expensive", "expensive", "cheap"], "and", "bit expensive"),
           Rule(308, ["ultra expensive", "ultra expensive", "cheap"], "and", "expensive"),
           
           Rule(309, ["ultra expensive", "ultra cheap", "bit cheap"], "and", "bit cheap"),
           Rule(310, ["ultra expensive", "cheap", "bit cheap"], "and", "medium"),
           Rule(311, ["ultra expensive", "bit cheap", "bit cheap"], "and", "medium"), 
           Rule(312, ["ultra expensive", "medium", "bit cheap"], "and", "bit expensive"),
           Rule(313, ["ultra expensive", "bit expensive", "bit cheap"], "and", "bit expensive"),
           Rule(314, ["ultra expensive", "expensive", "bit cheap"], "and", "bit expensive"),
           Rule(315, ["ultra expensive", "ultra expensive", "bit cheap"], "and", "expensive"),
           
           Rule(316, ["ultra expensive", "ultra cheap", "medium"], "and", "medium"),
           Rule(317, ["ultra expensive", "cheap", "medium"], "and", "medium"),
           Rule(318, ["ultra expensive", "bit cheap", "medium"], "and", "medium"), 
           Rule(319, ["ultra expensive", "medium", "medium"], "and", "bit expensive"),
           Rule(320, ["ultra expensive", "bit expensive", "medium"], "and", "bit expesive"),
           Rule(321, ["ultra expensive", "expensive", "medium"], "and", "expensive"),
           Rule(322, ["ultra expensive", "ultra expensive", "medium"], "and", "expensive"),
           
           Rule(323, ["ultra expensive", "ultra cheap", "bit expensive"], "and", "medium"),
           Rule(324, ["ultra expensive", "cheap", "bit expensive"], "and", "medium"),
           Rule(325, ["ultra expensive", "bit cheap", "bit expensive"], "and", "bit expensive"), 
           Rule(326, ["ultra expensive", "medium", "bit expensive"], "and", "bit expensive"),
           Rule(327, ["ultra expensive", "bit expensive", "bit expensive"], "and", "expensive"),
           Rule(328, ["ultra expensive", "expensive", "bit expensive"], "and", "expensive"),
           Rule(329, ["ultra expensive", "ultra expensive", "bit expensive"], "and", "expensive"),
           
           Rule(330, ["ultra expensive", "ultra cheap", "expensive"], "and", "medium"),
           Rule(331, ["ultra expensive", "cheap", "expensive"], "and", "bit expensive"),
           Rule(332, ["ultra expensive", "bit cheap", "expensive"], "and", "bit expensive"), 
           Rule(333, ["ultra expensive", "medium", "expensive"], "and", "expensive"),
           Rule(334, ["ultra expensive", "bit expensive", "expensive"], "and", "expensive"),
           Rule(335, ["ultra expensive", "expensive", "expensive"], "and", "expensive"),
           Rule(336, ["ultra expensive", "ultra expensive", "expensive"], "and", "ultra expensive"),
           
           Rule(337, ["ultra expensive", "ultra cheap", "ultra expensive"], "and", "bit expensive"),
           Rule(338, ["ultra expensive", "cheap", "ultra expensive"], "and", "bit expensive"),
           Rule(339, ["ultra expensive", "bit cheap", "ultra expensive"], "and", "expensive"), 
           Rule(340, ["ultra expensive", "medium", "ultra expensive"], "and", "expensive"),
           Rule(341, ["ultra expensive", "bit expensive", "ultra expensive"], "and", "expensive"),
           Rule(342, ["ultra expensive", "expensive", "ultra expensive"], "and", "ultra expensive"),
           Rule(343, ["ultra expensive", "ultra expensive", "ultra expensive"], "and", "ultra expensive"),
         ]


# brand/price categorie
rules_5 = [Rule(1, ["cat1", "ultra cheap"], "and", "ultra cheap"), 
           Rule(2, ["cat1", "cheap"], "and", "ultra cheap"),
           Rule(3, ["cat1", "bit cheap"], "and", "cheap"), 
           Rule(4, ["cat1", "medium"], "and", "bit cheap"),
           Rule(5, ["cat1", "bit expensive"], "and", "medium"), 
           Rule(6, ["cat1", "expensive"], "and", "bit expensive"),
           Rule(7, ["cat1", "ultra expensive"], "and", "expensive"), 
           Rule(8, ["cat2", "ultra cheap"], "and", "ultra cheap"), 
           Rule(9, ["cat2", "cheap"], "and", "cheap"),
           Rule(10, ["cat2", "bit cheap"], "and", "bit cheap"), 
           Rule(11, ["cat2", "medium"], "and", "medium"),
           Rule(12, ["cat2", "bit expensive"], "and", "bit expensive"), 
           Rule(13, ["cat2", "expensive"], "and", "expensive"),
           Rule(14, ["cat2", "ultra expensive"], "and", "ultra expensive"),            
           Rule(15, ["cat3", "ultra cheap"], "and", "cheap"), 
           Rule(16, ["cat3", "cheap"], "and", "cheap"),
           Rule(17, ["cat3", "bit cheap"], "and", "bit cheap"), 
           Rule(18, ["cat3", "medium"], "and", "bit cheap"),
           Rule(19, ["cat3", "bit expensive"], "and", "medium"), 
           Rule(20, ["cat3", "expensive"], "and", "medium"),
           Rule(21, ["cat3", "ultra expensive"], "and", "bit expensive"),           
           Rule(22, ["cat4", "ultra cheap"], "and", "cheap"), 
           Rule(23, ["cat4", "cheap"], "and", "bit cheap"),
           Rule(24, ["cat4", "bit cheap"], "and", "bit cheap"), 
           Rule(25, ["cat4", "medium"], "and", "medium"),
           Rule(26, ["cat4", "bit expensive"], "and", "medium"), 
           Rule(27, ["cat4", "expensive"], "and", "medium"),
           Rule(28, ["cat4", "ultra expensive"], "and", "bit expensive"), 
           Rule(29, ["cat5", "ultra cheap"], "and", "bit cheap"), 
           Rule(30, ["cat5", "cheap"], "and", "bit cheap"),
           Rule(31, ["cat5", "bit cheap"], "and", "medium"), 
           Rule(32, ["cat5", "medium"], "and", "medium"),
           Rule(33, ["cat5", "bit expensive"], "and", "medium"), 
           Rule(34, ["cat5", "expensive"], "and", "bit expensive"),
           Rule(35, ["cat5", "ultra expensive"], "and", "bit expensive"), 
           Rule(36, ["cat6", "ultra cheap"], "and", "bit cheap"), 
           Rule(37, ["cat6", "cheap"], "and", "medium"),
           Rule(38, ["cat6", "bit cheap"], "and", "medium"), 
           Rule(39, ["cat6", "medium"], "and", "medium"),
           Rule(40, ["cat6", "bit expensive"], "and", "bit expensive"), 
           Rule(41, ["cat6", "expensive"], "and", "bit expensive"),
           Rule(42, ["cat6", "ultra expensive"], "and", "expensive"),            
           Rule(43, ["cat7", "ultra cheap"], "and", "medium"), 
           Rule(44, ["cat7", "cheap"], "and", "medium"),
           Rule(45, ["cat7", "bit cheap"], "and", "medium"), 
           Rule(46, ["cat7", "medium"], "and", "bit expensive"),
           Rule(47, ["cat7", "bit expensive"], "and", "bit expensive"), 
           Rule(48, ["cat7", "expensive"], "and", "expensive"),
           Rule(49, ["cat7", "ultra expensive"], "and", "expensive"), 
           Rule(50, ["cat8", "ultra cheap"], "and", "medium"), 
           Rule(51, ["cat8", "cheap"], "and", "medium"),
           Rule(52, ["cat8", "bit cheap"], "and", "bit expensive"), 
           Rule(53, ["cat8", "medium"], "and", "bit expensive"),
           Rule(54, ["cat8", "bit expensive"], "and", "expensive"), 
           Rule(55, ["cat8", "expensive"], "and", "expensive"),
           Rule(56, ["cat8", "ultra expensive"], "and", "ultra expensive")       
        ]

In [8]:
#
# DEMO
#

# return firing strength for gearbox/horsepower    
def p2(a, b):  
    datapoints2 = [a, b]
    inputs2 = [gearbox, horsePower] 
    rulebase = Rulebase(rules_2)
    prijs2 = rulebase.calculate_firing_strengths(datapoints2, inputs2)
    return prijs2

# return firing strength for price/price/pice
def p4(a, b, c):
    datapoints4 = [a, b, c]
    inputs4 = [allPrices, allPrices, allPrices] 
    rulebase = Rulebase(rules_4)
    prijs4 = rulebase.calculate_firing_strengths(datapoints4, inputs4)
    return prijs4

from collections import *
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from collections import Counter

from IPython.display import display
def f(kilometers, gearbox, horsepower, fuel, vehicle, age, brands):
    # GET INPUT FROM SLIDER
    inputs1 = [kilometer, yearOfRegistration]
    inputs2 = [gearbox, horsePower]
    inputs3 = [fuelType, vehicleType]
    output = price
  
    # calculate firing strength for first model
    datapoints1 = [kilometers, age]
    rulebase = Rulebase(rules_1)
    prijs1 = rulebase.calculate_firing_strengths(datapoints1, inputs1)
    
    # calculate firing strength for second model
    prijs2 = p2(gearbox, horsepower);
    
    # calculate firing strength for third model
    datapoints3 = [fuel, vehicle]
    rulebase = Rulebase(rules_3)
    prijs3 = rulebase.calculate_firing_strengths(datapoints3, inputs3)

    # From these previous 3 firing strengths calculate next input for fourth model. 
    # Take highest strength and use as singular values.
    prijs1 = max(prijs1, key=prijs1.get)
    prijs2 = max(prijs2, key=prijs2.get)
    prijs3 = max(prijs3, key=prijs3.get)
    list = [prijs1, prijs2, prijs3]
    
    # Make singular values out of prize ranges
    consequents = []
    for e in list:
        if e == 'ultra cheap':
            e = 1
        if e == 'cheap':
            e = 2        
        if e == 'bit cheap':
            e = 3
        if e == 'medium':
            e = 4
        if e == 'bit expensive':
            e = 5            
        if e == 'expensive':
            e = 6
        if e == 'ultra expensive':
            e = 7
        consequents.append(e)
    
    # Combine previous inputs and return strength
    prijs4 = p4(consequents[0], consequents[1], consequents[2])

    # Make this strength a singular value
    from collections import Counter, OrderedDict
    class OrderedCounter(Counter, OrderedDict):
        pass 
    counterlist = OrderedCounter(prijs4)
    q=0
    for e in counterlist.keys():
            if e == 'ultra cheap':
                q = 1
            if e == 'cheap':
                q = 2        
            if e == 'bit cheap':
                q = 3
            if e == 'medium':
                q = 4
            if e == 'bit expensive':
                q = 5            
            if e == 'expensive':
                q = 6
            if e == 'ultra expensive':
                q = 7
      
    # Final model, return prize category
    datapoint = [brands, q]  
    inputs_last = [brand, allPrices]
    rulebase = Rulebase(rules_5)
    prijs = rulebase.calculate_firing_strengths(datapoint, inputs_last)
    for e in prijs.keys():
        p = e
    
    # Output category
    print("Price category for car:", e)
    return 

w = interactive(f, kilometers=(5000,149999,100), gearbox=(1,2,1),  horsepower=(21, 775, 1), 
                fuel=(1,4,1), vehicle=(1,5,1),
               age=(1910, 2018, 1), brands=(1,8,1));

display(w)

A Jupyter Widget

Gearbox 1 = automatik  
Gearbox 2 = manuell  

fuel 1 = Diesel  
fuel 2 = Benzin  
fuel 3 = lpg  
fuel 4 = cng  

vehicle 1 = kleinwagen  
vehicle 2 = combi  
vehicle 3 = bus  
vehicle 4 = suv  
vehicle 5 = limousin  

brands 1 = Daewoo & Lada & Trabant & daihatsu & Smart  
brands 2 = Suzuki & Saab & Subaru & Rover & Lancia  
brands 3 = Fiat  & Honda & Citroen & Chrysler & Renault  
brands 4 = Seat  & Toyota & Peugeot & Mazda & Mitsubishi  
brands 5 = Chevrolet  & Hyundai & Kia & Mini & Nissan  
brands 6 = Volvo  & Jeep & Ford & Alfa Romeo  
brands 7 = Jaguar & Land Rover & Volkwagen  
brands 8 = Mercedes & Porsche & Audi & Bmw

In [16]:
#
# Accuracy
#

# Calculate accuracy from data-set. Each price encountered in data-set belongs to a certain prize category, 
# match these with output categories from model. 
from collections import Counter
import csv
# read file
ifile = open('test_cars.csv', 'r')
reader = csv.reader(ifile)
prijs = 0
accuracy = 0
list_actual = []
list_prijs = []
for row in reader:
    # skip header
    if row[0]!= 'price':
        # skip empty prices
        if row[1] != "":
            # skip ranges outside of our discourse
            if int(row[0]) > 99999 or int(row[0]) < 500:
                continue;
            if int(row[4]) != 0:
                # make input for first model
                if int(row[5]) > 149999:
                    continue
                datapoint1 = [int(row[5]), int(row[2])]
                
                # make input for second model
                if int(row[4]) < 20 or int(row[4]) > 349:
                    continue
                if row[3]=='manuell':
                    datapoint2 = [2, int(row[4])]
                else :
                    datapoint2 = [1, int(row[4])]
                
                # make input for third model
                if row[6]=='diesel':           
                    if row[1]=='limousine':
                        datapoint3 = [1, 5]
                    elif row[1]=='bus':
                        datapoint3 = [1, 3]
                    elif row[1]=='kleinwagen':
                        datapoint3 = [1, 1]
                    elif row[1]=='kombi':
                        datapoint3 = [1, 2]
                    elif row[1]=='suv':
                        datapoint3 = [1, 4]
                    else:
                        continue
                elif row[6]=='benzin':
                    if row[1]=='limousine':
                        datapoint3 = [2, 5]
                    elif row[1]=='bus':
                        datapoint3 = [2, 3]
                    elif row[1]=='kleinwagen':
                        datapoint3 = [2, 1]
                    elif row[1]=='kombi':
                        datapoint3 = [2, 2]
                    elif row[1]=='suv':
                        datapoint3 = [2, 4]

                elif row[6]=='cng':
                    if row[1]=='limousine':
                        datapoint3 = [4, 5]
                    if row[1]=='bus':
                        datapoint3 = [4, 3]
                    if row[1]=='kleinwagen':
                        datapoint3 = [4, 1]
                    if row[1]=='kombi':
                        datapoint3 = [4, 2]
                    if row[1]=='suv':
                        datapoint3 = [4, 4]
                elif row[6]=='lpg':
                    if row[1]=='limousine':
                        datapoint3 = [3, 5]
                    if row[1]=='bus':
                        datapoint3 = [3, 3]
                    if row[1]=='kleinwagen':
                        datapoint3 = [3, 1]
                    if row[1]=='kombi':
                        datapoint3 = [3, 2]
                    if row[1]=='suv':
                        datapoint3 = [3, 4]


                inputs1 = [kilometer, yearOfRegistration]
                inputs2 = [gearbox, horsePower]
                inputs3 = [fuelType, vehicleType]
                output = price

                # calculate strength for first model
                rulebase = Rulebase(rules_1)      
                prijs1 = rulebase.calculate_firing_strengths(datapoint1, inputs1)

                # calculate strength for second model
                rulebase = Rulebase(rules_2)
                prijs2 = rulebase.calculate_firing_strengths(datapoint2, inputs2)

                # calculate strength for third model
                rulebase = Rulebase(rules_3)
                prijs3 = rulebase.calculate_firing_strengths(datapoint3, inputs3)
   
                # use max strength for every rule
                prijs1 = max(prijs1, key=prijs1.get)
                prijs2 = max(prijs2, key=prijs2.get)
                prijs3 = max(prijs3, key=prijs3.get)
                list = [prijs1, prijs2, prijs3]

                # make singular values for further computation
                consequents = []
                for e in list:
                    if e == 'ultra cheap':
                        e = 1
                    if e == 'cheap':
                        e = 2        
                    if e == 'bit cheap':
                        e = 3
                    if e == 'medium':
                        e = 4
                    if e == 'bit expensive':
                        e = 5            
                    if e == 'expensive':
                        e = 6
                    if e == 'ultra expensive':
                        e = 7
                    consequents.append(e)
                    
                # Calculate firing strength from these singular values, either 0 or 1
                prijs4 = p4(consequents[0], consequents[1], consequents[2])
                from collections import Counter, OrderedDict

                # Define singular values
                class OrderedCounter(Counter, OrderedDict):
                    pass 
                counterlist = OrderedCounter(prijs4)
                q=0
                for e in counterlist.keys():
                        if e == 'ultra cheap':
                            q = 1
                        elif e == 'cheap':
                            q = 2        
                        elif e == 'bit cheap':
                            q = 3
                        elif e == 'medium':
                            q = 4
                        elif e == 'bit expensive':
                            q = 5            
                        elif e == 'expensive':
                            q = 6
                        elif e == 'ultra expensive':
                            q = 7
                        else:
                            continue

                # Singular value of car brands together with singular value of price category 
                # make datapoint
                if (row[7]=='daewoo' or row[7]=='lada' or row[7]=='trabant' or row[7]=='daihatsu' or row[7]=='smart'):
                        datapoint = [1, q]
                elif (row[7]=='suzuki' or row[7]=='saab' or row[7]=='sabaru' or row[7]=='rover' or row[7]=='lancia'):
                        datapoint = [2, q]
                elif (row[7]=='fiat' or row[7]=='honda' or row[7]=='citroen' or row[7]=='chrysler' or row[7]=='renault'):
                        datapoint = [3, q]  
                elif (row[7]=='seat' or row[7]=='toyota' or row[7]=='peugeot' or row[7]=='mazda' or row[7]=='mitsubishi'):
                        datapoint = [4, q]  
                elif (row[7]=='chevrolet' or row[7]=='hyundai' or row[7]=='kia' or row[7]=='mini' or row[7]=='nissan'):
                        datapoint = [5, q]  
                elif (row[7]=='volvo' or row[7]=='jeep' or row[7]=='ford' or row[7]=='alfa_romeo'):
                        datapoint = [6, q]
                elif (row[7]=='mercedes_benz' or row[7]=='porsche' or row[7]=='audi' or row[7]=='bmw'):
                        datapoint = [8, q]  
                elif (row[7]=='jaguar' or row[7]=='land_rover' or row[7]=='volkswagen'):
                        datapoint = [7, q]
                else:
                    continue
                inputs= [brand, allPrices]
                rulebase = Rulebase(rules_5)
      
                # calculate firing strength (always 0 or 1) and return category
                strength = rulebase.calculate_firing_strengths(datapoint, inputs)
  
                from collections import Counter, OrderedDict
    
                # append category that model predicted
                for e in strength.keys():
                    list_prijs.append(e)

                # append category of price in data-set
                if int(row[0])<=3000:
                    list_actual.append('ultra cheap')
                elif int(row[0])> 3000 and int(row[0])<6000:
                    list_actual.append('cheap')
                elif int(row[0])>=6000 and int(row[0])<9000:
                    list_actual.append('bit cheap')

                elif int(row[0])>=9000 and int(row[0])<12000:
                    list_actual.append('medium')

                elif int(row[0])>=12000 and int(row[0])<180000:
                    list_actual.append('bit expensive')

                elif int(row[0])>=18000 and int(row[0])<280000:
                    list_actual.append('expensive')

                elif int(row[0])>=280000:
                    list_actual.append('ultra expensive')


# Compute accuracy
counter =0
for i in range(0, (len(list_actual)-1)):
    if (list_actual[i]==list_prijs[i]):
        counter = counter + 1
print("Accuracy is:", counter/len(list_actual))
print(list_actual.count('ultra expensive'),'actual count of ultra expensive')
print(list_prijs.count('ultra expensive'), 'count of model in ultra expensive')
print('---')
print(list_actual.count('expensive'),'actual count of expensive')
print(list_prijs.count('expensive'), 'count of model in expensive')
print('---')

print(list_actual.count('bit expensive'),'actual count of bit expensive')
print(list_prijs.count('bit expensive'), 'count of model in bit expensive')
print('---')
print(list_actual.count('medium'),'actual count of medium')
print(list_prijs.count('medium'), 'count of model in medium')
print('---')
print(list_actual.count('bit cheap'),'actual count of bit cheap')
print(list_prijs.count('bit cheap'), 'count of model in bit cheap')
print('---')
print(list_actual.count('cheap'),'actual count of cheap')
print(list_prijs.count('cheap'), 'count of model in cheap')
print('---')
print(list_actual.count('ultra cheap'),'actual count of ultra cheap')
print(list_prijs.count('ultra cheap'), 'count of model in ultra cheap')
print('---')


Accuracy is: 0.2894736842105263
0 actual count of ultra expensive
0 count of model in ultra expensive
---
0 actual count of expensive
67 count of model in expensive
---
268 actual count of bit expensive
243 count of model in bit expensive
---
68 actual count of medium
274 count of model in medium
---
120 actual count of bit cheap
120 count of model in bit cheap
---
144 actual count of cheap
42 count of model in cheap
---
160 actual count of ultra cheap
14 count of model in ultra cheap
---
